In [1]:
from sklearn.datasets import load_iris, load_diabetes
from sklearn.metrics import f1_score, r2_score, precision_score, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, recall_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import time

In [2]:
data = load_iris()
X = data.data
y = data.target
kn = KNeighborsClassifier(n_neighbors = 3)

scoring = {
    'f1_macro': make_scorer(f1_score, average='weighted'),
    'recall_macro': make_scorer(recall_score, average='weighted')
}
start1 = time.time()
score1 = cross_validate(kn, X, y, n_jobs = 4, verbose = 5, cv = 5, scoring = scoring, return_train_score=True)
end1 = time.time()

print(score1['test_f1_macro'].mean())
print(score1['test_recall_macro'].mean())

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


0.9664818612187034
0.9666666666666668


[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.1s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.8s finished


In [3]:
start2 = time.time()
score2 = cross_validate(kn, X, y, n_jobs = 4, verbose = 5, cv = 10, scoring = scoring, return_train_score=True)
end2 = time.time()
print(score2['test_f1_macro'].mean())
print(score2['test_recall_macro'].mean())

0.9659090909090908
0.9666666666666666


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   6 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished


In [4]:
print(score1['fit_time']) #czy to nie jest czas trenowania modelu? Uśrednić to?
train_timeS1 = end1 - start1
train_timeS2 = end2 - start2
print(train_timeS1)
print(train_timeS2)

for i, f1 in enumerate(score1['test_f1_macro'], 1):
    print(f"Fold {i} F1 Score: {f1}")

[0.00102544 0.00089669 0.00096846 0.00092626 0.00092411]
0.7693085670471191
0.0890817642211914
Fold 1 F1 Score: 0.9665831244778613
Fold 2 F1 Score: 0.9665831244778613
Fold 3 F1 Score: 0.9326599326599326
Fold 4 F1 Score: 0.9665831244778613
Fold 5 F1 Score: 1.0


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


scoring2 = {
    'f1_macro': make_scorer(f1_score, average='macro'),
    'precision_macro': make_scorer(precision_score, average='macro', zero_division=0)
}

rf = RandomForestClassifier(random_state=42)
parameters = {
    'max_depth': [1, 2, 4, 8, 16, 32, 64],
    'n_estimators': list(range(1, 26))
}

start3 = time.time()
mod = GridSearchCV(rf, parameters, cv = 5, scoring = scoring2, refit='precision_macro', verbose = 1, n_jobs = -1)
mod.fit(X,y)
end3 = time.time()
cv_results = mod.cv_results_


cv_results['mean_test_f1_macro'].mean()

[CV] END  f1_macro: (train=0.967, test=0.967) recall_macro: (train=0.967, test=0.967) total time=   0.0s
[CV] END  f1_macro: (train=0.956, test=1.000) recall_macro: (train=0.956, test=1.000) total time=   0.0s
[CV] END  f1_macro: (train=0.956, test=1.000) recall_macro: (train=0.956, test=1.000) total time=   0.0s
[CV] END  f1_macro: (train=0.958, test=0.967) recall_macro: (train=0.958, test=0.967) total time=   0.0s
[CV] END  f1_macro: (train=0.950, test=1.000) recall_macro: (train=0.950, test=1.000) total time=   0.0s
[CV] END  f1_macro: (train=0.956, test=1.000) recall_macro: (train=0.956, test=1.000) total time=   0.0s
[CV] END  f1_macro: (train=0.956, test=1.000) recall_macro: (train=0.956, test=1.000) total time=   0.0s
[CV] END  f1_macro: (train=0.958, test=0.967) recall_macro: (train=0.958, test=0.967) total time=   0.0s
[CV] END  f1_macro: (train=0.963, test=0.933) recall_macro: (train=0.963, test=0.933) total time=   0.0s
[CV] END  f1_macro: (train=0.970, test=0.933) recall_ma

/home/maciejrudy/conda/envs/pandas130/lib/python3.9/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


0.9557569893635829

In [6]:
print("F1 Scores for each parameter combination:")
'''
for params, f1 in zip(cv_results['params'], cv_results['mean_test_f1_macro']):
    print(f"Params: {params}, F1 Score: {f1}")

'''

F1 Scores for each parameter combination:


'\nfor params, f1 in zip(cv_results[\'params\'], cv_results[\'mean_test_f1_macro\']):\n    print(f"Params: {params}, F1 Score: {f1}")\n\n'

In [7]:
print(cv_results['mean_test_precision_macro'].mean())
print("Precision score for each parameter combination:")
for params, f1 in zip(cv_results['params'], cv_results['mean_test_precision_macro']):
    print(f"Params: {params}, F1 Score: {f1}")

0.9596944017863965
Precision score for each parameter combination:
Params: {'max_depth': 1, 'n_estimators': 1}, F1 Score: 0.47799802536644637
Params: {'max_depth': 1, 'n_estimators': 2}, F1 Score: 0.7648574335880838
Params: {'max_depth': 1, 'n_estimators': 3}, F1 Score: 0.9724941724941726
Params: {'max_depth': 1, 'n_estimators': 4}, F1 Score: 0.9724941724941726
Params: {'max_depth': 1, 'n_estimators': 5}, F1 Score: 0.9632996632996633
Params: {'max_depth': 1, 'n_estimators': 6}, F1 Score: 0.9707070707070707
Params: {'max_depth': 1, 'n_estimators': 7}, F1 Score: 0.9707070707070707
Params: {'max_depth': 1, 'n_estimators': 8}, F1 Score: 0.9634343434343435
Params: {'max_depth': 1, 'n_estimators': 9}, F1 Score: 0.946868686868687
Params: {'max_depth': 1, 'n_estimators': 10}, F1 Score: 0.9707070707070707
Params: {'max_depth': 1, 'n_estimators': 11}, F1 Score: 0.9707070707070707
Params: {'max_depth': 1, 'n_estimators': 12}, F1 Score: 0.9707070707070707
Params: {'max_depth': 1, 'n_estimators': 1

In [13]:
cv_results

{'mean_fit_time': array([0.00565205, 0.00797973, 0.0120378 , 0.01508322, 0.01835294,
        0.01855249, 0.01829939, 0.01859813, 0.01893749, 0.021487  ,
        0.02226291, 0.02532258, 0.02754288, 0.0280345 , 0.03118148,
        0.03577065, 0.04030724, 0.04149871, 0.04366355, 0.04321613,
        0.04531288, 0.04964085, 0.05882206, 0.05696735, 0.05479321,
        0.00422568, 0.00604043, 0.00804105, 0.01216455, 0.01237602,
        0.01535177, 0.01621404, 0.02252932, 0.02126946, 0.02774806,
        0.03044782, 0.02717714, 0.03405933, 0.03923893, 0.0344152 ,
        0.03789601, 0.03843336, 0.03930044, 0.04460449, 0.04683805,
        0.04636574, 0.04779701, 0.04942307, 0.05320973, 0.05483727,
        0.00418696, 0.00621004, 0.00801635, 0.01211171, 0.0142715 ,
        0.01409154, 0.01764455, 0.02155423, 0.022755  , 0.02557311,
        0.02760592, 0.02738047, 0.03052588, 0.03457747, 0.03566251,
        0.0375545 , 0.03680258, 0.04038162, 0.0430841 , 0.04403906,
        0.05789194, 0.05505023,

In [9]:
print("Best Configuration:", mod.best_params_)
print("Best Precision Score:", mod.best_score_)

Best Configuration: {'max_depth': 1, 'n_estimators': 3}
Best Precision Score: 0.9724941724941726


In [10]:
start4 = time.time()
mod2 = GridSearchCV(rf, parameters, cv = 10, scoring = scoring2, refit='precision_macro', verbose = 1, n_jobs = -1)
mod2.fit(X,y)
end4 = time.time()
cv_results2 = mod2.cv_results_
print("Best Configuration:", mod2.best_params_)
print("Best Precision Score:", mod2.best_score_)

Fitting 10 folds for each of 175 candidates, totalling 1750 fits
Best Configuration: {'max_depth': 8, 'n_estimators': 3}
Best Precision Score: 0.9722222222222223


In [11]:
print("Time for cv = 5: ", end3 - start3)
print("Time for cv = 10: ", end4 - start4)

Time for cv = 5:  5.7123799324035645
Time for cv = 10:  8.953128576278687


In [21]:
diabetes = load_diabetes()
X3 = diabetes.data
y3 = diabetes.target
kn3 = KNeighborsClassifier(n_neighbors = 5)

scoring3 = {
    'R2': make_scorer(r2_score),
    'mean_absolute_error': make_scorer(mean_absolute_error)
}

start3 = time.time()
score3 = cross_validate(kn3, X3, y3, n_jobs = 2, verbose = 5, cv = 5, scoring = scoring3, return_train_score=True)
end3 = time.time()
time3 = end3 - start3

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
/home/maciejrudy/conda/envs/pandas130/lib/python3.9/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.3s finished


So StratifiedKFold will try to keep that ratio in each fold. Now you have specified 10 folds (n_splits). So that means in a single fold, for class 'y' to maintain the ratio, at least 2 / 10 = 0.2 members. But we cannot give less than 1 member (sample) so that's why its throwing an error there.

If instead of n_splits=10, you have set n_splits=2, then it would have worked, because than the number of members for 'y' will be 2 / 2 = 1. For n_splits = 10 to work correctly, you need to have atleast 10 samples for each of your classes.

In [31]:
start4 = time.time()
score4 = cross_validate(kn3, X3, y3, n_jobs = 2, verbose = 15, cv = 10, scoring = scoring3, return_train_score=True)
end4 = time.time()
time4 = end4 - start4

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


ValueError: n_splits=10 cannot be greater than the number of members in each class.

In [33]:
print(score3)
print("Time spent: ", time3)
print(y3.shape)

{'fit_time': array([0.00687194, 0.00459909, 0.00384378, 0.0054791 , 0.00352383]), 'score_time': array([0.04230857, 0.02993011, 0.02790642, 0.04150605, 0.02106833]), 'test_R2': array([-0.46851491, -0.30685643, -0.83681986, -0.55930103, -0.42704228]), 'train_R2': array([-0.31080892, -0.21825669, -0.36288554, -0.16903963, -0.39352396]), 'test_mean_absolute_error': array([68.08988764, 69.39325843, 82.36363636, 79.35227273, 74.23863636]), 'train_mean_absolute_error': array([65.04249292, 62.9121813 , 65.48587571, 60.56214689, 66.3700565 ])}
Time spent:  0.34302616119384766
(442,)


In [29]:
print("test_R2", score3["test_R2"])
print("test_R2", score3["test_R2"].mean())

print("test_mean_absolute_error", score3["test_mean_absolute_error"])
print("test_mean_absolute_error", score3["test_mean_absolute_error"].mean())

print("Time spent: ", time3)

test_R2 [-0.46851491 -0.30685643 -0.83681986 -0.55930103 -0.42704228]
test_R2 -0.5197069019332347
test_mean_absolute_error [68.08988764 69.39325843 82.36363636 79.35227273 74.23863636]
test_mean_absolute_error 74.68753830439223


In [ ]:
print("test_R2", score4["test_R2"])
print("test_R2", score4["test_R2"].mean())

print("test_mean_absolute_error", score4["test_mean_absolute_error"])
print("test_mean_absolute_error", score4["test_mean_absolute_error"].mean())

print("Time spent: ", time4)

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
import numpy as np
import time

# Load diabetes dataset
data = load_diabetes()
X, y = data.data, data.target

# Define Random Forest model
rf = RandomForestRegressor(random_state=42)

# Define parameter grid
param_distributions = {
    'max_depth': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
    'n_estimators': list(range(3, 82, 3))
}

# Define scoring metrics
scoring = {
    'r2': make_scorer(r2_score),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

# A: Perform RandomizedSearchCV with 100 combinations
start_time = time.time()
random_search = RandomizedSearchCV(
    rf, param_distributions, n_iter=100, cv=5, scoring=scoring, refit='r2', n_jobs=-1, verbose=1
)
random_search.fit(X, y)
end_time = time.time()

# B: Best configuration
print("Best Configuration:", random_search.best_params_)
print("Best R^2 Score:", random_search.best_score_)

# C: Average R^2 score
cv_results = random_search.cv_results_
r2_scores = cv_results['mean_test_r2']
print("Average R^2 Score:", np.mean(r2_scores))

# D: R^2 score for each fold
print("R^2 Scores for each parameter combination:")
for params, r2 in zip(cv_results['params'], r2_scores):
    print(f"Params: {params}, R^2 Score: {r2}")

# E: MAE for each fold
mae_scores = -cv_results['mean_test_mae']  # Negative because MAE was negated for scoring
print("MAE Scores for each parameter combination:")
for params, mae in zip(cv_results['params'], mae_scores):
    print(f"Params: {params}, MAE Score: {mae}")
print("Average MAE:", np.mean(mae_scores))

# F: 10-Fold Cross-Validation
random_search_10 = RandomizedSearchCV(
    rf, param_distributions, n_iter=200, cv=10, scoring=scoring, refit='r2', n_jobs=-1, verbose=1
)
random_search_10.fit(X, y)
print("Best Configuration with 10 folds:", random_search_10.best_params_)
print("Best R^2 Score with 10 folds:", random_search_10.best_score_)

# G: Training time
print("Training time:", end_time - start_time, "seconds")


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/maciejrudy/conda/envs/pandas130/lib/python3.9/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Configuration: {'n_estimators': 42, 'max_depth': 4}
Best R^2 Score: 0.43624561695224917
Average R^2 Score: 0.40431569846605764
R^2 Scores for each parameter combination:
Params: {'n_estimators': 42, 'max_depth': 32}, R^2 Score: 0.4181608610324103
Params: {'n_estimators': 21, 'max_depth': 16}, R^2 Score: 0.4101437742761461
Params: {'n_estimators': 78, 'max_depth': 128}, R^2 Score: 0.4202360274140894
Params: {'n_estimators': 30, 'max_depth': 32}, R^2 Score: 0.4123216452922168
Params: {'n_estimators': 39, 'max_depth': 256}, R^2 Score: 0.4177378965850468
Params: {'n_estimators': 24, 'max_depth': 16}, R^2 Score: 0.4107350989446865
Params: {'n_estimators': 12, 'max_depth': 512}, R^2 Score: 0.37738059513564676
Params: {'n_estimators': 81, 'max_depth': 1024}, R^2 Score: 0.4207626243416828
Params: {'n_estimators': 81, 'max_depth': 16}, R^2 Score: 0.423300384025893
Params: {'n_estimators': 33, 'max_depth': 2}, R^2 Score: 0.40314448651182266
Params: {'n_estimators': 9, 'max_depth': 128}, R^2

/home/maciejrudy/conda/envs/pandas130/lib/python3.9/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Configuration with 10 folds: {'n_estimators': 78, 'max_depth': 4}
Best R^2 Score with 10 folds: 0.41746405674263726
Training time: 20.21255874633789 seconds
